<a href="https://colab.research.google.com/github/Nick088Official/WhiteRabbitNeo-7b-v1.5a-Google-Colab/blob/main/WhiteRabbitNeo_7b_v1_5a_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WhiteRabbitNeo-7b-v1.5a AI Model WEB UI

Model Used: https://huggingface.co/WhiteRabbitNeo/WhiteRabbitNeo-7B-v1.5a

Google Colab Made by [Nick088](https://linktr.ee/nick088)

In [ ]:
#@title Install & Load Dependencies, Model

#@markdown Use GPU (faster, max 12 free hours daily limit): Set the Video Card from Edit -> Notebook Settings -> T4 GPU OR Any other GPUs based on your Google Colab Subscription

!pip install einops
!pip install accelerate
!pip install huggingface_hub
import torch
from huggingface_hub import login
from IPython.display import clear_output

if torch.cuda.is_available():
    device = "cuda"
    print("Using GPU")
else:
    device = "cpu"
    print("Using CPU")

# Now you can use 'device' for your PyTorch operations

torch.set_default_device(device)

#@markdown The normal version is the official one (PAID GPU REQUIRED), the other ones are GGUF quantized, compressed to consume less ram, see more info about them in the [Hugging Face repo of WhiteRabbitNeo-7b-v1.5a GGUF](https://huggingface.co/bartowski/WhiteRabbitNeo-7B-v1.5a-GGUF)

whiterabbitneo_model_version = "Q2_K" #@param ['normal', 'Q2_K', 'Q3_K_S', 'Q3_K_M', 'Q3_K_L', 'Q4_0', 'Q4_K_M', 'Q4_K_S', 'Q5_0', 'Q5_K_S', 'Q5_K_M', 'Q6_K', 'Q8_0']

#@markdown Only for the normal version: Go to https://huggingface.co/settings/tokens and make a token with the read role, and paste it here.
Hugging_Face_Read_Token = "" #@param {type:"string"}

# Authenticate
login(token=Hugging_Face_Read_Token)

if whiterabbitneo_model_version == "normal":
  from transformers import AutoModelForCausalLM, AutoTokenizer
  model = AutoModelForCausalLM.from_pretrained("WhiteRabbitNeo/WhiteRabbitNeo-7B-v1.5a", torch_dtype="auto", trust_remote_code=True)
  tokenizer = AutoTokenizer.from_pretrained("WhiteRabbitNeo/WhiteRabbitNeo-7B-v1.5a", trust_remote_code=True)
else:
  if device == "cuda":
    !CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
  else:
    !pip install llama-cpp-python
  from huggingface_hub import hf_hub_download
  from llama_cpp import Llama
  model_name = "bartowski/WhiteRabbitNeo-7B-v1.5a-GGUF"
  model_file = f"WhiteRabbitNeo-7B-v1.5a-{whiterabbitneo_model_version}.gguf"
  model_path = hf_hub_download(model_name,
                               filename=model_file,
                               local_dir='/content')
  from llama_cpp import Llama
  llm = Llama(model_path=model_path,
            n_gpu_layers=-1)

clear_output()
print(f"Done downloading WhiteRabbitNeo-7b {whiterabbitneo_model_version}!")

In [ ]:
#@title Run UI

def answer(user_prompt, system_prompt, max_new_tokens, repetition_penalty):
  if whiterabbitneo_model_version == "normal":
    # Concatenate system and user prompts
    full_prompt = system_prompt + user_prompt

    # Tokenize the combined prompt
    inputs = tokenizer(full_prompt, return_tensors="pt", return_attention_mask=False)

    # Generate text based on the combined prompt and additional inputs
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        repetition_penalty=repetition_penalty
    )

    # Extract the generated text from the model output
    text = tokenizer.batch_decode(outputs)[0]
    text = text[len(system_prompt):].strip()
    return text
  else:
    response = llm(full_prompt, max_tokens=max_new_tokens, repeat_penalty=repetition_penalty)
    response = response['choices'][0]['text']
    return response

# Define the gradio interface
interface = gr.Interface(
    fn=answer,
    inputs=[
        gr.Textbox(lines=2, label="Your Prompt"),
        gr.Textbox(lines=2, label="System Prompt"),
        gr.Slider(value=3000, minimum=256, maximum=8192, step=1, interactive=True, label="Max Tokens"),
        gr.Slider(value=1.2, minimum=0.0, maximum=2.0, step=0.1, interactive=True, label="Repetition Penalty"),
    ],
    outputs="text",
    title=f" WhiteRabbitNeo 7b v1.5a {whiterabbitneo_model_version}",
)


# Launch the gradio interface
interface.launch(share=True, debug=True)